In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
import geopandas

%matplotlib inline

In [2]:
df = pd.read_csv('./data/df_with_tags.gz')

In [3]:
df['Review_Date'] = df['Review_Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))

In [106]:
df['Review_Month'] = df.Review_Month.apply(lambda x: str(x))

### Config

In [353]:
tmp = []
for i in df.Diff:
    if i < -2:
        tmp.append('Worse')
    elif i < -0.5:
        tmp.append('Bad')
    elif i < 0.5:
        tmp.append('Average')
    elif i >= 0.5:
        tmp.append('Good')

df['Diff_Recode'] = tmp

In [354]:
y_col = 'Diff_Recode'
x_col = ['Review_Month', 'City', 'Pet', 'Purpose', 'Whom', 'Room_Recode', 'Nationality_Recode', 'Length_Recode',
         'Average_Score', 'Total_Number_of_Reviews_Reviewer_Has_Given']

#### R-Squared Formula

In [355]:
from scipy.stats import linregress

def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return r_value**2

### Start

In [412]:
df2 = df.sample(n=500, random_state=1)

In [413]:
X = df2[x_col]
y = df2[y_col]

In [414]:
X = X.fillna('Not Available')

In [415]:
X = pd.get_dummies(X, prefix_sep='_', drop_first=True)

In [416]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

### Gradient Boosting

In [417]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

clf = RandomForestClassifier(n_estimators = 1000, max_depth = 5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
accuracy_score(pred, y_test), cohen_kappa_score(pred, y_test)

(0.44666666666666666, 0.01744140162575958)

### Random Forest

In [418]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
accuracy_score(pred, y_test), cohen_kappa_score(pred, y_test)

(0.36666666666666664, -0.01771175546350534)

### XGBoost

In [419]:
import xgboost as xgb

clf = xgb.XGBClassifier(n_estimators = 100)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
accuracy_score(pred, y_test), cohen_kappa_score(pred, y_test)

(0.31333333333333335, -0.030000000000000027)

In [402]:
#from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, plot_confusion_matrix